In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.misc import imread, imsave, imresize

trainLabel = pd.read_csv("trainLabels.csv")

x_image_paths = [str(i) + ".png" for i in range(1, 50001)]
x_data = []
for path in x_image_paths:
    x_data.append(imread("train/" + path)) #scipy의 imread로 이미지를 불러들이면 numpy배열로 값을 반환한다.

x_data = np.array(x_data) / 256

y_data = trainLabel["label"]
y_data = y_data.replace(["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"], 
              [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]) #정답 레이블 단어를 숫자 인덱스에 맞도록 치환

y_data = y_data.values.reshape((-1, 1))

# plt.imshow(x_data[0]) #이미지 출력
#x_data[0].shape (32, 32, 3)

In [3]:
test_image_path = [str(i) + ".png" for i in range(1, 300001)]
test_data = []
for path in test_image_path:
    test_data.append(imread("test/" + path)) #scipy의 imread로 이미지를 불러들이면 numpy배열로 값을 반환한다.

test_data = np.array(test_data) / 256

In [4]:
def get_batches_train(x_arr, y_arr):
    # Get the batch size and number of batches we can make
    batch_size = 500
    n_batches = len(x_arr)//batch_size
    
    for n in range(0, len(x_arr), batch_size):
        # The features
        x = x_arr[n:n+batch_size]
        # The targets, shifted by one
        y = y_arr[n:n+batch_size]
        yield x, y

In [5]:
def get_batches_test(arr):
    # Get the batch size and number of batches we can make
    batch_size = 500
    n_batches = len(arr)//batch_size
    
    for n in range(0, len(arr), batch_size):
        # The features
        x = arr[n:n+batch_size]
        yield x

In [6]:
X = tf.placeholder(tf.float32, shape=[None, 32, 32, 3]) #인풋
Y = tf.placeholder(tf.int32, shape=[None, 1]) #정답레이블
keep_prob = tf.placeholder(tf.float32) #드랍아웃 파라미터

W = tf.Variable(tf.truncated_normal((5, 5, int(X.get_shape()[-1]), 50), stddev=0.1))
b = tf.Variable(tf.random_normal([50]))
L = tf.nn.conv2d(X, W, strides=[1, 1, 1, 1], padding="SAME") #필터 가중치 적용
L = tf.nn.bias_add(L, b)
L = tf.nn.relu(L)
L = tf.nn.max_pool(L, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W1 = tf.Variable(tf.truncated_normal((5, 5, int(L.get_shape()[-1]), 128), stddev=0.1))
b1 = tf.Variable(tf.random_normal([128]))
L1 = tf.nn.conv2d(L, W1, strides=[1, 1, 1, 1], padding="SAME") #필터 가중치 적용
L1 = tf.nn.bias_add(L1, b1)
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L1 = tf.nn.dropout(L1, keep_prob=0.7) #드랍아웃

W2 = tf.Variable(tf.truncated_normal((5, 5, int(L1.get_shape()[-1]), 256), stddev=0.1))
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding="SAME") #필터 가중치 적용
L2 = tf.nn.bias_add(L2, b2)
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.nn.dropout(L2, keep_prob=0.7) #드랍아웃

conv = tf.nn.dropout(L2, keep_prob=0.7) #드랍아웃

flat = tf.reshape(conv, [-1, int(conv.get_shape()[1]) * int(conv.get_shape()[2]) * int(conv.get_shape()[3])])

W_fc1 = tf.Variable(tf.truncated_normal((int(flat.get_shape()[-1]), 256), stddev=0.1)) #가중치
b_fc1 = tf.Variable(tf.random_normal([256])) #편향
fc1 = tf.nn.relu(tf.matmul(flat, W_fc1) + b_fc1)
fc1 = tf.nn.dropout(fc1, keep_prob=0.7)

W_fc2 = tf.Variable(tf.truncated_normal((int(fc1.get_shape()[-1]), 128), stddev=0.1)) #가중치
b_fc2 = tf.Variable(tf.random_normal([128])) #편향
fc2 = tf.nn.relu(tf.matmul(fc1, W_fc2) + b_fc2)
fc2 = tf.nn.dropout(fc2, keep_prob=0.7)

W_fc3 = tf.Variable(tf.truncated_normal((int(fc2.get_shape()[-1]), 64), stddev=0.1)) #가중치
b_fc3 = tf.Variable(tf.random_normal([64])) #편향
fc3 = tf.nn.relu(tf.matmul(fc2, W_fc3) + b_fc3)
fc3 = tf.nn.dropout(fc3, keep_prob=0.7)

W_fc = tf.Variable(tf.truncated_normal((int(fc3.get_shape()[-1]), 10), stddev=0.1)) #가중치
b_fc = tf.Variable(tf.random_normal([10])) #편향

out = tf.matmul(fc3, W_fc) + b_fc

Y_one_hot = tf.one_hot(Y, 10)
Y_one_hot = tf.reshape(Y_one_hot, [-1, 10])

# tf.reset_default_graph() #그래프 초기화

logits = tf.identity(out) #결과값
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)) #손실
optimizer = tf.train.AdamOptimizer().minimize(cost) #Adam으로 최적화

hypothesis = tf.nn.softmax(logits)
prediction = tf.argmax(hypothesis, 1) #예측된 값에서 가장 높은 값을 가지는 요소의 인덱스 추출

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y_one_hot, 1))
#결과값의 가장높은 확률(소프트맥스이므로)의 인덱스와 정답레이블의 인덱스 같은 지 비교
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [8]:
pred_arr = []

with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    counter = 0

    for step in range(100):
        for x, y in get_batches_train(x_data, y_data):
            counter += 1
            session.run(optimizer, feed_dict={X: x, Y: y, keep_prob: 0.7}) #optimizer에 모든 텐서가 연결되어 있으므로 optimizer 실행
            loss = session.run(cost, feed_dict={X: x, Y: y, keep_prob: 0.7})
            if counter%50 == 0:
                print("Step: {:5}\tTraining Step: {}\tLoss: {:.3f}".format(step, counter, loss))

    for x in get_batches_test(test_data):
        pred = session.run(prediction, feed_dict={X: x, keep_prob: 1}) 
        pred_arr.extend(pred)

Step:     0	Training Step: 50	Loss: 2.508
Step:     0	Training Step: 100	Loss: 2.499
Step:     1	Training Step: 150	Loss: 2.357
Step:     1	Training Step: 200	Loss: 2.245
Step:     2	Training Step: 250	Loss: 2.215
Step:     2	Training Step: 300	Loss: 2.161
Step:     3	Training Step: 350	Loss: 2.127
Step:     3	Training Step: 400	Loss: 2.049
Step:     4	Training Step: 450	Loss: 2.020
Step:     4	Training Step: 500	Loss: 1.988
Step:     5	Training Step: 550	Loss: 1.820
Step:     5	Training Step: 600	Loss: 1.866
Step:     6	Training Step: 650	Loss: 1.747
Step:     6	Training Step: 700	Loss: 1.820
Step:     7	Training Step: 750	Loss: 1.680
Step:     7	Training Step: 800	Loss: 1.801
Step:     8	Training Step: 850	Loss: 1.658
Step:     8	Training Step: 900	Loss: 1.662
Step:     9	Training Step: 950	Loss: 1.591
Step:     9	Training Step: 1000	Loss: 1.580
Step:    10	Training Step: 1050	Loss: 1.554
Step:    10	Training Step: 1100	Loss: 1.616
Step:    11	Training Step: 1150	Loss: 1.501
Step:   

In [9]:
pred_arr

[4,
 0,
 1,
 8,
 8,
 5,
 0,
 2,
 3,
 3,
 6,
 6,
 7,
 8,
 6,
 0,
 4,
 4,
 0,
 6,
 5,
 8,
 8,
 1,
 2,
 1,
 9,
 3,
 5,
 2,
 2,
 7,
 3,
 6,
 4,
 3,
 3,
 5,
 8,
 8,
 6,
 4,
 9,
 4,
 7,
 1,
 4,
 4,
 3,
 1,
 3,
 7,
 3,
 6,
 8,
 8,
 7,
 8,
 2,
 2,
 4,
 5,
 4,
 6,
 8,
 3,
 8,
 7,
 3,
 2,
 9,
 4,
 9,
 3,
 3,
 7,
 0,
 3,
 4,
 3,
 3,
 4,
 0,
 4,
 4,
 6,
 5,
 6,
 7,
 9,
 1,
 0,
 5,
 7,
 9,
 6,
 4,
 2,
 9,
 4,
 4,
 0,
 5,
 3,
 8,
 0,
 1,
 6,
 6,
 6,
 4,
 7,
 4,
 0,
 9,
 7,
 9,
 7,
 0,
 2,
 8,
 9,
 7,
 2,
 8,
 0,
 0,
 5,
 4,
 5,
 9,
 7,
 6,
 6,
 2,
 8,
 5,
 3,
 7,
 1,
 6,
 5,
 8,
 4,
 9,
 1,
 6,
 8,
 3,
 6,
 2,
 5,
 5,
 8,
 9,
 7,
 0,
 8,
 3,
 6,
 3,
 3,
 6,
 5,
 6,
 1,
 6,
 6,
 4,
 6,
 4,
 6,
 3,
 6,
 2,
 0,
 3,
 1,
 2,
 1,
 6,
 9,
 4,
 6,
 8,
 5,
 8,
 9,
 7,
 0,
 1,
 8,
 0,
 6,
 9,
 2,
 1,
 3,
 8,
 5,
 8,
 6,
 0,
 6,
 0,
 2,
 3,
 2,
 6,
 3,
 1,
 7,
 7,
 6,
 3,
 9,
 5,
 8,
 3,
 1,
 6,
 0,
 5,
 4,
 0,
 6,
 7,
 3,
 7,
 9,
 8,
 4,
 4,
 4,
 7,
 6,
 4,
 4,
 9,
 0,
 1,
 3,
 4,
 3,
 3,
 7,
 2,
 9,
 7,
 8,


In [10]:
result = pd.Series(pred_arr) 
result = result.replace([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 
                        ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]) #정답 레이블 단어를 숫자 인덱스에 맞도록 치환
result.index += 1

In [11]:
result.to_csv(r'pandas.csv', header=None, index=True, sep=',', mode='a')